In [1]:
import sys
sys.path.insert(0, "../../../")

In [2]:
import deeptorch as dtm

import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl

In [3]:
# Load the MNIST dataset
mnist = torchvision.datasets.MNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)

mnist_test = torchvision.datasets.MNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)

In [4]:
mnist_dataloader = torch.utils.data.DataLoader(mnist, batch_size=32, num_workers=4)
mnist_test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=32, num_workers=4)

# Train using default ImageClassifier

In [6]:
classifier = dtm.ImageClassifier(num_classes=10).build((1, 1, 28, 28))

classifier

c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


ImageClassifier(
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (classifier): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Lin

# Ways to modify the classifier

The following are a few ways to modify the classifier.

In [7]:
classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dict(
        channels_out=[8, 16, 32]
    ),
        
)
classifier.build((1, 1, 28, 28))

classifier

ImageClassifier(
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (classifier): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (1): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linea

In [8]:
blocks = [
    dtm.ConvPoolBlock(
        channels_out=16,
        conv=dtm.ConvActNormBlock(channels_out=16, conv=dict(padding=0))
    ),
    dtm.ConvPoolBlock(
        channels_out=32,
        conv=dtm.ConvActNormBlock(channels_out=32, conv=dict(padding=0))
    ),
    dtm.ConvPoolBlock(
        channels_out=64,
        conv=dtm.ConvActBlock(channels_out=64, conv=dict(padding=0))
    )
]

classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dict(
        blocks=blocks
    ),
        
)
classifier.build((1, 1, 28, 28))

classifier

ImageClassifier(
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (classifier): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
        )
        (1): MaxPool2d(kernel_size=2, s

In [9]:
class GlobalAvgPoolConnector(dtm.LazyModule):
    def __init__(self):
        super().__init__()
        
    def build(self, *args):
        return nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.LazyLinear(32),
            nn.ReLU(),
            nn.LazyLinear(32),
            nn.ReLU()
        )

blocks = [
    dtm.ConvPoolBlock(
        channels_out=16,
        conv=dtm.ConvActNormBlock(channels_out=16, conv=dict(padding=0))
    ),
    dtm.ConvPoolBlock(
        channels_out=32,
        conv=dtm.ConvActNormBlock(channels_out=32, conv=dict(padding=0))
    ),
    dtm.ConvActBlock(
        channels_out=64,
        conv=dtm.ConvActBlock(channels_out=64, conv=dict(padding=0))
    )
]

backbone = dtm.Encoder2d(
    blocks=blocks
)


classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=backbone,
    connector=GlobalAvgPoolConnector()
)
classifier.build((1, 1, 28, 28))

classifier

ImageClassifier(
  (val_accuracy): MulticlassAccuracy()
  (test_accuracy): MulticlassAccuracy()
  (classifier): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Sequential(
          (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
          (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
          (1): ReLU()
        )
        (1): ReLU()
      )
    )
    (

# Training the classifier

In [10]:
trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | val_accuracy  | MulticlassAccuracy | 0     
1 | test_accuracy | MulticlassAccuracy | 0     
2 | classifier    | Sequential         | 26.9 K
-----------------------------------------------------
26.9 K    Trainable params
0         Non-trainable params
26.9 K    Total params
0.107     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [11]:
trainer.test(dataloaders=mnist_test_dataloader)

c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:134: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_6\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_6\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9815999865531921     │
│         test_loss         │     1.479982614517212     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9815999865531921, 'test_loss': 1.479982614517212}]